# Precinct Matching Framework

In [1]:
import pandas as pd
import geopandas as gpd
from pprint import pprint

from op_verification.reference_data import (
    geoid_to_county_name,
    state_to_fips,
    state_to_state_po,
    state_abbreviation_to_state_name,
    state_fip_to_county_to_geoid,
)

### Import the datasets

In [2]:
county_id = 'Chambers County'

county_results_filename = 'election_results_county_id={}.csv'.format(county_id)
county_results_df = pd.read_csv(county_results_filename)
county_shapefile_filename = 'shapefile_county_id={}'.format(county_id)
county_shapefile_gdf = gpd.read_file(county_shapefile_filename)

# correct for the truncation caused by 10 character column name limit in shapefiles
county_shapefile_gdf.rename(columns={'original_p':'original_precinct_name'}, inplace=True)

The next cell aliases `county_results_df` as `df` and `county_shapefile_gdf` as `gdf` here because typing fewer characters allows for faster data exploration. 

In [3]:
df = county_results_df.copy()
gdf = county_shapefile_gdf.copy()

In [4]:
df.head()

,county,precinct,Gov_DEM,Gov_REP,LtGov_DEM,LtGov_REP,StHOR_DEM,StHOR_IND,StHOR_LIB,StHOR_REP,StSen_DEM,StSen_IND,StSen_REP,SP_DEM,SP_LIB,SP_REP,USHOR_DEM,USHOR_REP,county_id,original_precinct_name
0,Chambers County,ABSENTEE,96.0,261.0,97.0,254.0,106.0,NaN,NaN,250.0,100.0,NaN,255.0,67.0,NaN,104.0,102.0,255.0,Chambers County,ABSENTEE
1,Chambers County,BUFFALO,73.0,139.0,78.0,132.0,75.0,NaN,NaN,136.0,76.0,NaN,135.0,65.0,NaN,40.0,75.0,136.0,Chambers County,BUFFALO
2,Chambers County,CUSSETA,77.0,170.0,77.0,168.0,91.0,NaN,NaN,150.0,74.0,NaN,169.0,56.0,NaN,63.0,79.0,167.0,Chambers County,CUSSETA
3,Chambers County,FIVE POINTS,121.0,63.0,123.0,59.0,120.0,NaN,NaN,64.0,123.0,NaN,59.0,92.0,NaN,21.0,121.0,63.0,Chambers County,FIVE POINTS
4,Chambers County,FREDONIA,75.0,239.0,83.0,226.0,76.0,NaN,NaN,236.0,85.0,NaN,226.0,50.0,NaN,86.0,79.0,235.0,Chambers County,FREDONIA


In [5]:
gdf.head()

,STATEFP,COUNTYFP,VTDST,NAMELSAD,VTDI,LSAD,CHNG_TYPE,ORIG_NAME,ORIG_CODE,RELATE,NAME,VINTAGE,FUNCSTAT,JUSTIFY,MTFCC,county_id,original_precinct_name,geometry
0,01,017,000001,Standing Rock Comm Ctr,A,00,None,None,None,None,Standing Rock Comm Ctr,90,N,None,G5240,Chambers County,Standing Rock Comm Ctr,"POLYGON Z ((-85.30681 33.10825 0.00000, -85.29..."
1,01,017,000002,Mt Olive Club Hse,A,00,None,None,None,None,Mt Olive Club Hse,90,N,None,G5240,Chambers County,Mt Olive Club Hse,"POLYGON Z ((-85.46533 33.06997 0.00000, -85.46..."
2,01,017,000003,Milltown Voting Station,A,00,None,None,None,None,Milltown Voting Station,90,N,None,G5240,Chambers County,Milltown Voting Station,"POLYGON Z ((-85.56554 33.06840 0.00000, -85.56..."
3,01,017,000004,Union Hill Comm Club Hse,A,00,None,None,None,None,Union Hill Comm Club Hse,90,N,None,G5240,Chambers County,Union Hill Comm Club Hse,"POLYGON Z ((-85.59345 33.00012 0.00000, -85.59..."
4,01,017,000005,Five Points Club Hse,A,00,None,None,None,None,Five Points Club Hse,90,N,None,G5240,Chambers County,Five Points Club Hse,"POLYGON Z ((-85.43819 33.03329 0.00000, -85.43..."


### Check Preconditions
These should all pass - they're here to ensure that everything in `setup.ipynb` worked correctly

In [6]:
# TODO: Pass the precondition described above which takes the form of an assert statement in this cell.
assert 'county_id' in df.columns and 'county_id' in gdf.columns
assert 'original_precinct_name' in df.columns and 'original_precinct_name' in gdf.columns

### General Modifications
Its normally benificial to apply some modifications uniformly to all precincts. For example, its good practice to make everything lower case. This modification is made in `edit_precinct_name` - read its specification to learn more about how to use it to make more modifications. Consider removing substrings that appear in every precinct name like `voting district`. Be careful of removing words that will result in duplicate precinct names. For example, if there are two `Newtown Voting District` precincts in the shapefile, and the election results have `Newtown Boro` and `Newtown Township`, its okay to remove `Voting District`, but you probably don't want to remove `Boro` or `Township`. Of course, this will differ from County to County - so be vigilant!

In [7]:
dataset_name_df = 'df'
dataset_name_gdf = 'gdf'

def edit_precinct_name(prec_name, 
    remove_lst=[], 
    target_to_replacement={},
    stopping_words=[],
    prec_dict={}):
    '''
    Returns a lower case precinct name (string) with certian modifications depending other arguments. 
    
    Modifications are performed in order of the parameters they depend on. By convention, case is 
    ignored by making prec_name lower case. Accordingly, one should pass arguements with lower case
    elements. That is, keys of the dictionaries and elements of lists should be lower case strings.

	Parameters:
		prec_name (str): precinct name
		remove_lst ((str) list): if a string in this list is a substring in prec_name it will be removed. 
            All elements should be lower case.
        target_to_replacement ({str:str} dictionary): keys (targets) will be replaced with their 
            corresponding value (replacements) in prec_name. All keys should be lower case.
        stopping_words ({str} list): If any substring of prec_name contains a element of stopping_words
             that is adjacent to a space character it will be removed. All elements should be lower case.
        prec_dict ({str:str} dictionary): After all the modifications above, if the edited prec_name
            string is in the set of keys for prec_dict, then it will be replaced with that key's value.  
            All keys should be lower case.

	Returns:
		prec_name (str): prec_name arguement returned with the 
    '''
    prec_name = str(prec_name).lower()
    for word in remove_lst:
        prec_name = prec_name.replace(word, '')
    for target, replacement in target_to_replacement.items():
        prec_name = prec_name.replace(target, replacement)
    words = prec_name.split()
    words = [word.lstrip('0') for word in words if word not in stopping_words]
    prec_name = " ".join(words)
    return prec_dict[prec_name] if prec_name in prec_dict.keys() else prec_name

df_to_gdf = {
 'buffalo': 'buffalo/lebanon church',
 'cusseta': 'cusseta fire and rescue',
 'five points': 'five points club hse',
 'fredonia': 'fredonia fire and rescue',
 'huguley': 'huguley fire station',
 'lafayette': 'lafayette hs',
 'lakeview': 'lakeview fire and rescue',
 'lanett': 'lanette hs gym',
 'milltown': 'milltown voting station',
 'mt_ olive': 'mt olive club hse',
 'penton': 'penton fire and rescue',
 'ridge grove': 'ridge grove fire and rescue',
 'sparkling springs': 'sparkling springs comm ctr',
 'standing rock': 'standing rock comm ctr',
 'stroud': 'stroud comm hse',
 'union hill': 'union hill comm club hse',
 'waverly': 'waverly old school',
 'west chambers': 'west chambers voting station'
}

gdf_to_df = {
}

# Tune the matching by adding optional arguements to edit
df['edited_precinct_name'] = df['original_precinct_name'].apply(lambda name: edit_precinct_name(name,prec_dict=df_to_gdf))
gdf['edited_precinct_name'] = gdf['original_precinct_name'].apply(lambda name: edit_precinct_name(name,prec_dict=gdf_to_df))

######## Manual Corrections ###########
# Make precinct specific corrections here like splitting one precinct into two because of new congressional districts
# e.g gdf.loc[gdf['VTDST']=='000290','edited_precinct_name'] = 'howard township'

# make the 'original_precinct_name, edited_precinct_name' for use in the loop below
df['original_precinct_name, edited_precinct_name'] = df[['original_precinct_name','edited_precinct_name']].apply(tuple, axis=1)
gdf['original_precinct_name, edited_precinct_name'] = gdf[['original_precinct_name','edited_precinct_name']].apply(tuple, axis=1)

######## Matching Framework ###########
unmatched_precinct_lst_gdf = []
unmatched_precinct_lst_df = ['absentee', 'provisional']

precinct_list_df = sorted(list(df[df['county_id'] == county_id]['original_precinct_name, edited_precinct_name'].unique()), key=lambda x: x[1])
precinct_list_gdf = sorted(list(gdf[gdf['county_id'] == county_id]['original_precinct_name, edited_precinct_name'].unique()), key=lambda x: x[1])

precinct_set_df = {x[1] for x in precinct_list_df if x[1] not in unmatched_precinct_lst_df}
precinct_set_gdf = {x[1] for x in precinct_list_gdf if x[1] not in unmatched_precinct_lst_gdf}

unmatched_precincts_df = sorted(list(precinct_set_df - precinct_set_gdf))
unmatched_precincts_gdf = sorted(list(precinct_set_gdf - precinct_set_df))
n_unmatched = len(unmatched_precincts_df) + len(unmatched_precincts_gdf)
if n_unmatched > 0:
        print("county_id: '{}' | {} precincts in {} | {} precincts in {}:\n".format(county_id, len(precinct_list_df), dataset_name_df, len(precinct_list_gdf), dataset_name_gdf))
        n_precincts_total = len(precinct_list_df) + len(precinct_list_gdf)
        print(n_unmatched, " precincts are unmatched out of ", n_precincts_total)
        df_unmatched = df[(df['edited_precinct_name'].isin(unmatched_precincts_df)) & (df.county_id == county_id)]
        gdf_unmatched = gdf[(gdf['edited_precinct_name'].isin(unmatched_precincts_gdf)) & (gdf.county_id == county_id)]
        if n_unmatched > 100:
            print("\nLook for parterns and use change the parameters to edit_precinct_name accordingly.\n")
            for index, (original_precinct_name_df, edited_precinct_name_df) in enumerate(precinct_list_df):
                original_precinct_name_gdf, edited_precinct_name_gdf = precinct_list_gdf[index]
                if edited_precinct_name_df in unmatched_precincts_df and edited_precinct_name_gdf in unmatched_precincts_gdf:
                    print("{} <-- {} ({})".format(edited_precinct_name_df, original_precinct_name_df, dataset_name_df))
                    print("{} <-- {} ({})\n".format(edited_precinct_name_gdf, original_precinct_name_gdf, dataset_name_gdf))
        else:
            print("unmatched_precincts_df ({}) - len = {}| '{}':".format(dataset_name_df, len(unmatched_precincts_df), county_id), unmatched_precincts_df)
            print("\nunmatched_precincts_gdf ({}) - len = {}| '{}':".format(dataset_name_gdf, len(unmatched_precincts_gdf), county_id), unmatched_precincts_gdf)
            precinct_modification_dictionary_df_to_gdf = {unmatched_precincts_df[i]: unmatched_precincts_gdf[i] if i < len(unmatched_precincts_gdf) else '' for i in range(len(unmatched_precincts_df))}
            precinct_modification_dicitonary_gdf_to_df = {unmatched_precincts_gdf[i]: unmatched_precincts_df[i] if i < len(unmatched_precincts_df) else '' for i in range(len(unmatched_precincts_gdf))}
            print("{}  to {} precinct modification dictionary: ".format(dataset_name_df, dataset_name_gdf))
            print("'{}':".format(county_id))
            pprint(precinct_modification_dictionary_df_to_gdf)
            print("{}  to {} precinct modification dictionary: ".format(dataset_name_gdf, dataset_name_df))
            print("'{}':".format(county_id))
            pprint(precinct_modification_dicitonary_gdf_to_df)
            for index, (original_precinct_name_df, edited_precinct_name_df) in enumerate(precinct_list_df):
                original_precinct_name_gdf, edited_precinct_name_gdf = precinct_list_gdf[index]
                if edited_precinct_name_df in unmatched_precincts_df or edited_precinct_name_gdf in unmatched_precincts_gdf:
                    print("{} <-- {} ({})".format(edited_precinct_name_df, original_precinct_name_df, dataset_name_df))
                    print("{} <-- {} ({})\n".format(edited_precinct_name_gdf, original_precinct_name_gdf, dataset_name_gdf))
            else:
                print("Add unmatched precincts to the unmatched precinct.")
else:
    print("All Done! (make sure you have one to one matches)")

All Done! (make sure you have one to one matches)


### Validate

In [8]:
acceptable_duplicates_to_reason_df = {
}
counts = df['edited_precinct_name'].value_counts()
duplicates = counts[(counts>1) & (~counts.index.isin(acceptable_duplicates_to_reason_df.keys()))]
assert len(duplicates) == 0

In [9]:
acceptable_duplicates_to_reason_gdf = {
}
counts = gdf['edited_precinct_name'].value_counts()
duplicates = counts[(counts>1) & (~counts.index.isin(acceptable_duplicates_to_reason_gdf.keys()))]
assert len(duplicates) == 0

In [10]:
gdf.edited_precinct_name.nunique()

19

In [11]:
gdf.original_precinct_name.nunique()

19

In [12]:
df.edited_precinct_name.nunique()

21

In [13]:
df.edited_precinct_name.nunique()

21

In [14]:
df = df[~df['edited_precinct_name'].isin(unmatched_precinct_lst_df)]

### Export

In [15]:
df['edited_precinct_name'] = df['edited_precinct_name'].str.title()
df['loc, prec'] = df.apply(lambda row: county_id + ', ' + row['edited_precinct_name'], axis=1)
df = df.groupby(by='loc, prec').sum()

In [16]:
gdf['edited_precinct_name'] = gdf['edited_precinct_name'].str.title()
gdf['loc, prec'] = gdf.apply(lambda row: county_id + ', ' + row['edited_precinct_name'], axis=1)
gdf = gdf.dissolve(by='loc, prec', as_index=False)
county_to_geoid = state_fip_to_county_to_geoid[1]
gdf['GEOID'] = gdf['county_id'].map(county_to_geoid)
gdf = gdf.set_index('loc, prec')

In [17]:
cols = [ 'loc, prec', 'GEOID', 'county_id', 'edited_precinct_name'] + list(df.columns) + ['geometry']
cols

['loc, prec',
 'GEOID',
 'county_id',
 'edited_precinct_name',
 'Gov_DEM',
 'Gov_REP',
 'LtGov_DEM',
 'LtGov_REP',
 'StHOR_DEM',
 'StHOR_IND',
 'StHOR_LIB',
 'StHOR_REP',
 'StSen_DEM',
 'StSen_IND',
 'StSen_REP',
 'SP_DEM',
 'SP_LIB',
 'SP_REP',
 'USHOR_DEM',
 'USHOR_REP',
 'geometry']

In [18]:
joined_df = df.join(gdf, lsuffix='_left', rsuffix='_right').reset_index()
print(joined_df.shape)
joined_df.head(2)

(19, 38)


,"loc, prec",Gov_DEM,Gov_REP,LtGov_DEM,LtGov_REP,StHOR_DEM,StHOR_IND,StHOR_LIB,StHOR_REP,StSen_DEM,...,NAME,VINTAGE,FUNCSTAT,JUSTIFY,MTFCC,county_id,original_precinct_name,edited_precinct_name,"original_precinct_name, edited_precinct_name",GEOID
0,"Chambers County, Buffalo/Lebanon Church",73.0,139.0,78.0,132.0,75.0,0.0,0.0,136.0,76.0,...,Buffalo/Lebanon Church,90,N,None,G5240,Chambers County,Buffalo/Lebanon Church,Buffalo/Lebanon Church,"(Buffalo/Lebanon Church, buffalo/lebanon church)",01017
1,"Chambers County, Cusseta Fire And Rescue",77.0,170.0,77.0,168.0,91.0,0.0,0.0,150.0,74.0,...,Cusseta Fire and Rescue,90,N,None,G5240,Chambers County,Cusseta Fire and Rescue,Cusseta Fire And Rescue,"(Cusseta Fire and Rescue, cusseta fire and res...",01017


In [19]:
output = gpd.GeoDataFrame(joined_df[cols].rename(columns={'edited_precinct_name':'precinct'}))
output.head()

,"loc, prec",GEOID,county_id,precinct,Gov_DEM,Gov_REP,LtGov_DEM,LtGov_REP,StHOR_DEM,StHOR_IND,...,StHOR_REP,StSen_DEM,StSen_IND,StSen_REP,SP_DEM,SP_LIB,SP_REP,USHOR_DEM,USHOR_REP,geometry
0,"Chambers County, Buffalo/Lebanon Church",01017,Chambers County,Buffalo/Lebanon Church,73.0,139.0,78.0,132.0,75.0,0.0,...,136.0,76.0,0.0,135.0,65.0,0.0,40.0,75.0,136.0,"POLYGON Z ((-85.51579 32.93663 0.00000, -85.51..."
1,"Chambers County, Cusseta Fire And Rescue",01017,Chambers County,Cusseta Fire And Rescue,77.0,170.0,77.0,168.0,91.0,0.0,...,150.0,74.0,0.0,169.0,56.0,0.0,63.0,79.0,167.0,"POLYGON Z ((-85.39822 32.84452 0.00000, -85.39..."
2,"Chambers County, Five Points Club Hse",01017,Chambers County,Five Points Club Hse,121.0,63.0,123.0,59.0,120.0,0.0,...,64.0,123.0,0.0,59.0,92.0,0.0,21.0,121.0,63.0,"POLYGON Z ((-85.43819 33.03329 0.00000, -85.43..."
3,"Chambers County, Fredonia Fire And Rescue",01017,Chambers County,Fredonia Fire And Rescue,75.0,239.0,83.0,226.0,76.0,0.0,...,236.0,85.0,0.0,226.0,50.0,0.0,86.0,79.0,235.0,"POLYGON Z ((-85.33287 32.94292 0.00000, -85.33..."
4,"Chambers County, Huguley Fire Station",01017,Chambers County,Huguley Fire Station,365.0,954.0,354.0,934.0,378.0,0.0,...,931.0,361.0,0.0,943.0,225.0,0.0,334.0,374.0,942.0,"POLYGON Z ((-85.32965 32.87889 0.00000, -85.32..."


In [20]:
parent_dir = '../../data/shapefiles/county_level_precinct_election_results'
output.to_file('/'.join([parent_dir, "_".join(county_id.split())]))